In [11]:
from sklearn.model_selection import train_test_split
import pandas as pd
import collections
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import gensim
from sklearn.decomposition import TruncatedSVD
from gensim.models import word2vec
from gensim.models import KeyedVectors
import numpy as np
from sklearn.decomposition import PCA
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from pymagnitude import Magnitude, MagnitudeUtils

%cd ./data
from func1 import *

[Errno 2] No such file or directory: './data'
/mnt/data1/home/yoshida/my_research/data


In [2]:
SEED = 42

df = pd.read_csv("oversample_text.csv",encoding="utf-8",engine="python")  # データの読み込み
#df_clean = makeclean_csv("oversample_text")

# ラベルと文章を分ける
sentences = df.text.values
labels = df.tag.values
print(labels)

[0 0 0 ... 2 2 2]


In [3]:
#mecabによる形態素解析(ipadic-neologd),taggerを変えれば何でもok
def tokenize(text):
    cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
    path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,shell=True).communicate()[0]).decode('utf-8')
    m = MeCab.Tagger("-d {0}".format(path))  #mecab-ipadic-neologd
    #m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
    node = m.parse(text)
    node = node.split("\n")
    text_tokenize = []
    for i in range(len(node)):
        word = node[i].split("\t")
        if word[0] == "EOS":
            break
        text_tokenize.append(word[0])
    
    return text_tokenize

tokenized_sentence = []
for sentence in sentences:
    sen = tokenize(sentence)
    tokenized_sentence.append(sen)

In [4]:
#model = gensim.models.KeyedVectors.load("./chive-1.1-mc90_gensim/chive-1.1-mc90.kv")
model = Magnitude("https://sudachi.s3-ap-northeast-1.amazonaws.com/chive/chive-1.1-mc90-aunit.magnitude")
#model.query(word)

In [5]:
model2 = word2vec.Word2Vec(sentences=tokenized_sentence, vector_size=300, window=5, min_count=1, epochs=10)
#model.wx["word"]
#model.save("./output.model")

In [6]:
def avg_feature_vector(sentence, model, num_features):
    words = tokenize(sentence)
    feature_vec = np.zeros((num_features,), dtype="float32") # 特徴ベクトルの入れ物を初期化
    for word in words:
        feature_vec = np.add(feature_vec, model.query(word))
    if len(words) > 0:
        feature_vec = np.divide(feature_vec, len(words))
    return feature_vec

feature_vec_lis = []
for i,sentence in enumerate(sentences):
    feature_vec = avg_feature_vector(sentence,model,300)
    feature_vec_lis.append(feature_vec)
    
#print(feature_vec_lis[2])

In [7]:
#ベクトル化したものを圧縮し、数次元のデータにする
#主成分分析により２次元に圧縮する
pca = PCA(n_components=3)
pca.fit(feature_vec_lis)
data_pca= pca.transform(feature_vec_lis)

In [8]:
#df_3d = pd.DataFrame(np.concatenate([data_pca, y], axis=1), columns=['first component', 'second component','third component' 'tags']).astype({'first component':float, 'second component':float,'third component':float})

In [12]:
#3値分類におけるロジスティック回帰
# 7:3に学習データとテストデータを分割する
X_train = data_pca
y_train = labels
df2 = pd.read_csv("test_data.csv",encoding="utf-8",engine="python")  # データの読み込み
test_sentences = df2.text.values
y_test = df2.tag.values

#testデータに対してベクトル化処理
feature_vec_lis = []
for i,sentence in enumerate(test_sentences):
    feature_vec = avg_feature_vector(sentence,model,300)
    feature_vec_lis.append(feature_vec)
pca = PCA(n_components=3)
pca.fit(feature_vec_lis)
X_test = pca.transform(feature_vec_lis)

lr = LogisticRegression(random_state=SEED, n_jobs=-1)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

df_sub = pd.DataFrame(cm,columns=['Predicted safety', 'Predicted flaming','Predicted spam'], index=['Actual safety', 'Actual flaming','Actual spam'])
print(df_sub)
print("Accuracy : {}".format(accuracy_score(y_test,y_pred)))
print("Precision: {}".format(precision_score(y_test,y_pred,average = None)))
print("Recall : {}".format(recall_score(y_test,y_pred, average = None)))
print("F1-Score: {}".format(f1_score(y_test,y_pred,average = None)))


                Predicted safety  Predicted flaming  Predicted spam
Actual safety                 87                 61              65
Actual flaming                40                 88              25
Actual spam                    3                  2              29
Accuracy : 0.51
Precision: [0.66923077 0.58278146 0.24369748]
Recall : [0.4084507  0.5751634  0.85294118]
F1-Score: [0.50728863 0.57894737 0.37908497]
              precision    recall  f1-score   support

         0.0       0.67      0.41      0.51       213
         1.0       0.58      0.58      0.58       153
         2.0       0.24      0.85      0.38        34

    accuracy                           0.51       400
   macro avg       0.50      0.61      0.49       400
weighted avg       0.60      0.51      0.52       400



In [ ]:
print(classification_report(y_test,y_pred))